In [137]:
!pip install transformers
!pip install regex requests

In [0]:
import pandas as pd
import transformers
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Conv2D
import numpy as np
from transformers import RobertaTokenizer
from transformers import *
import tokenizers
from keras.preprocessing.sequence import pad_sequences
import torch
import torch.nn as nn

In [139]:
# Data Loader
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/tweet-sentiment-extraction/'
code = "4/zgHVgt5EYIxQpCKdQTmi61ujDfdn_p673NBszogLYWsCZMJpSbY_WIQ"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [140]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

def read_test():
  path = root_path + "test.csv"
  df = pd.read_csv(path)
  df['text'] = df['text'].astype(str)
  return shuffle(df,random_state=np.random.randint(0,10))
def reaf_submission():
    test=pd.read_csv('../input/tweet-sentiment-extraction/sample_submission.csv')
    return shuffle(test,random_state=np.random.randint(0,10))
def read_train():
  path = root_path + "train.csv"
  df = pd.read_csv(path)
  df['text'] = df['text'].astype(str)
  df['selected_text']=df['selected_text'].astype(str)
  return shuffle(df,random_state=np.random.randint(0,10))

print(read_train().sample(frac=1).head(20).index.tolist)
train_csv, test = read_train().sample(frac=1).reset_index(drop=True), read_test().sample(frac=1).reset_index(drop=True)
train, val = train_test_split(train_csv, test_size=0.12)

<bound method IndexOpsMixin.tolist of Int64Index([ 2346, 18279,  5558, 25447, 19862,  9863, 18583,  3120,  1004,
            16481, 14545, 13135,  8611,  9428, 25073,  5531, 22491, 10788,
            16982, 20782],
           dtype='int64')>


In [0]:
# Metric tested 
def jaccard(str1,str2):
  a = set(str1.lower().split()) 
  b = set(str2.lower().split())
  c = a.intersection(b)
  return float(len(c)) / (len(a) + len(b) - len(c))

In [142]:
currModel = 'roberta-base'
# using both to simplfy various methods
tokenzier = RobertaTokenizer.from_pretrained('roberta-base',do_lower_case=True)
tokenzier.save_vocabulary(root_path)
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file=root_path+'vocab.json', 
    merges_file=root_path+'merges.txt', 
    lowercase=True,
    add_prefix_space=True
)
# these are the ids used by our tokenizer
sentiment = {
    'neutral': int(tokenizer.encode('neutral').ids[0]),
    'positive': int(tokenizer.encode('positive').ids[0]),
    'negative':int(tokenizer.encode('negative').ids[0])
}
print(sentiment)

{'neutral': 7974, 'positive': 1313, 'negative': 2430}


In [0]:
# The structure of our input would be of the following form [Text={Context},special_token,sentiment={Question}]
# which is then predicted with selected_text.
def get_input_ids(idf):
    input_ids = []
    for i in range(idf.shape[0]):
        context,label = idf['text'].iloc[i].split(),idf['sentiment'].iloc[i].split()
        encoded_input = tokenzier.encode_plus(
                            " ".join(context)," ".join(label),                   # Sentence to encode.
                            # Add '[CLS]' and '[SEP]' 
                    )
        # encode_input returns input_ids and attention masks. 
        # Masks would be helpful latter to instruct model to avoid 
        # special tokens such as padding. 
        input_ids.append(encoded_input)
    return input_ids

In [0]:

train_set_ids = get_input_ids(train)
val_set_ids = get_input_ids(val)
test_ids = get_input_ids(test_csv)


The next part of data pre-processing is to come up with start and end tokens which gives start index and end index of the selected word in our encoded input. 

For instance, Conside the sample with

text = "Sooo SAD have to leave boston"

selected_text = "Sooo SAD" 

In this case start token is S and end token is D 
input_ids = [0, 3, 3, 11990, 560, 38457, 3, 2, 2, 2430, 2]

start_token = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

end_token = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

Note: There are two ways this can be done character level and token level. I'm using character level tokenization in order to prevent loss of training samples.



In [0]:
# print(input_ids[3])
def create_labels(train2,t_ids):
    st = []
    et = []
    for i in range(train2.shape[0]):
        text,label = " ".join(train2['text'].iloc[i].split()), " ".join(train2['selected_text'].iloc[i].split())
        char = np.zeros((len(text)))
        idx = text.find(label)
        char[idx:idx+len(label)] = 1
        if text[idx-1]==' ': char[idx-1] = 1
        enc = tokenizer.encode(text)
        offsets = enc.offsets
        chard_to_token_index = []
        start = list(char).index(1)+1
        end_idx = start+len(label)
        # print(start,end_idx)
        for k in range(len(offsets)):
            my_set = range(offsets[k][0],offsets[k][1])
            # print(my_set)
            if(start <= offsets[k][1] and  offsets[k][1] <= end_idx or (start in my_set and end_idx in my_set)):
                chard_to_token_index.append(k)
        start_tokens,end_tokens = [0]*len(t_ids[i].input_ids),[0]*len(t_ids[i].input_ids)
        if(len(chard_to_token_index) > 1):
            # print(i)
            start_tokens[chard_to_token_index[0]+1] = 1
            end_tokens[chard_to_token_index[1]+1] = 1
            st.append(start_tokens)
            et.append(end_tokens)
        else:
            start_tokens[chard_to_token_index[0]+1],end_tokens[chard_to_token_index[0]+1] = 1,1
            st.append(start_tokens)
            et.append(end_tokens)
    return st,et
train_st,train_et = create_labels(train,train_set_ids)
val_st,val_et = create_labels(val,val_set_ids)
# test_st,test_et = create_labels(test_csv,train_set_ids)

In [146]:
# Given the sequence length lets set MAX_LEN to 100
MAX_LEN = 100
def padding_input(ids,max_len,mask=False):
    print('\nPadding token: "{:}", ID: {:}'.format(tokenzier.pad_token, tokenzier.pad_token_id))
# seperate attention_masks and input_ids.
    print(ids[0])
    train_ids = [seq.input_ids for seq in ids]
    attention_masks = [seq.attention_mask for seq in ids]
    # input_ids = None
    train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long",value=1, truncating="post", padding="post")
    if mask:
        attention_masks = pad_sequences(attention_masks, maxlen=MAX_LEN, dtype="long", 
                            value=0, truncating="post", padding="post")
        
        return (train_ids,attention_masks)
    return train_ids
train_ids, train_masks = padding_input(train_set_ids,MAX_LEN,True)
val_ids,val_masks = padding_input(val_set_ids,MAX_LEN,True)
test_ids,test_masks = padding_input(test_ids,MAX_LEN,True)
# Converting to torch tensors in order to use GPU.


Padding token: "<pad>", ID: 1
{'input_ids': [0, 939, 3996, 169, 33175, 219, 350, 203, 28, 2802, 352, 14798, 361, 4197, 698, 452, 8, 939, 524, 45, 2602, 9, 14, 754, 4, 30016, 5998, 2, 2, 2430, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Padding token: "<pad>", ID: 1
{'input_ids': [0, 18134, 29411, 939, 11269, 139, 939, 376, 11, 1029, 329, 5, 3778, 1682, 164, 639, 5, 10722, 53, 122, 24, 34, 283, 124, 4005, 11, 127, 2931, 122, 15, 5, 1929, 29784, 2, 2, 7974, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Padding token: "<pad>", ID: 1
{'input_ids': [0, 14223, 2841, 119, 821, 1942, 12846, 2677, 3209, 3204, 11, 545, 360, 4, 939, 524, 98, 2283, 4, 2, 2, 1313, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [0]:
train_data = {
    "ids": torch.tensor(train_ids,dtype=torch.float),
    "att_mask": torch.tensor(train_masks,dtype=torch.float),
    "token_type_ids": torch.zeros((train.shape[0],MAX_LEN),dtype=torch.float)
}
val_data = {
    "ids": torch.tensor(val_ids,dtype=torch.float),
    "att_mask": torch.tensor(val_masks,dtype=torch.float),
    "token_type_ids": torch.zeros((val.shape[0],MAX_LEN),dtype=torch.float)

}
test_data = {
    "ids": torch.tensor(test_ids,dtype=torch.float),
    "att_mask": torch.tensor(test_masks,dtype=torch.float),
    "token_type_ids": torch.zeros((test.shape[0],MAX_LEN),dtype=torch.float)
}
train_label = {
    "start_tokens":torch.tensor(pad_sequences(train_st,maxlen=MAX_LEN, dtype="long", 
                            value=0, truncating="post", padding="post"),dtype=torch.float),
    "end_tokens": torch.tensor(pad_sequences(train_et,maxlen=MAX_LEN, dtype="long", 
                            value=0, truncating="post", padding="post"),dtype=torch.float),
}
val_label = {
    "start_tokens":torch.tensor(pad_sequences(val_st,maxlen=MAX_LEN, dtype="long", 
                            value=0, truncating="post", padding="post"),dtype=torch.float),
    "end_tokens": torch.tensor(pad_sequences(val_et,maxlen=MAX_LEN, dtype="long", 
                            value=0, truncating="post", padding="post"),dtype=torch.float),
}


In [148]:
# Create the DataLoader for our training set.
print(val_data["ids"].shape,val_data["att_mask"].shape,val_data["token_type_ids"].shape)
print(val_label["start_tokens"].shape,val_label["end_tokens"].shape)

torch.Size([3298, 100]) torch.Size([3298, 100]) torch.Size([3298, 100])
torch.Size([3298, 100]) torch.Size([3298, 100])


In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset
batch_size = 64

train_data = TensorDataset(train_data["ids"], train_data["att_mask"], train_data["token_type_ids"],train_label["start_tokens"],train_label["end_tokens"])
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# Create the DataLoader for our validation set.
validation_data = TensorDataset(val_data["ids"], val_data["att_mask"], val_data["token_type_ids"],val_label["start_tokens"],val_label["end_tokens"])
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)



In [0]:
# Data pre-processing is one of the most important tasks when it comes to NLP. Next includes model 
# selection and training in order to make prediction on this new task.
# Model
from transformers import RobertaModel,RobertaConfig
class SentimentAnalysis(nn.Module):
    def __init__(self,configs):
        super(SentimentAnalysis,self).__init__()
        self.roberta = RobertaModel.from_pretrained(root_path+'roberta-base-weights.bin',config=configs)
        self.container = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Conv1d(768,768,kernel_size=1)  
        )
        self.avgpool  =  nn.AvgPool1d(kernel_size=5)
        self.linear = nn.Linear(153,2)
        self.linear2 = nn.Linear(2,1)
    def forward(self,token_ids,att_masks,type_ids):
        outputs[0] = self.roberta(
            input_ids=token_ids,
            attention_mask=att_masks,
            token_type_ids=type_ids,
                        )
        bs = roberta.shape[0]   
        sl = self.container(torch.transpose(bs,dim0=1,dim1=2))
        avgpool = self.avgpool(torch.transpose(sq,dim0=1,dim1=2))
        start_logits = self.linear2(nn.functional.softmax(self.linear(avgpool),dim=2))
        el = self.container(torch.transpose(bs,dim0=1,dim1=2))
        avgpool = self.avgpool(self.avgpool(torch.transpose(el,dim0=1,dim1=2)))
        end_logits = self.linear2(nn.function.softmax(self.linear(avgpool),dim=2))
        start_logits = nn.functional.softmax(torch.flatten(start_logits,start_dim=1,end_dim=2),dim=1)
        end_logits = nn.functional.softmax(torch.flatten(start_logits,start_dim=1,end_dim=2),dim=1)
        return start_logits,end_logits

In [0]:
model = SentimentAnalysis(configs=root_path+'config.json')

In [152]:
from torchsummary import summary
device = torch.device("cuda")
model.to(device)
# print(model)

SentimentAnalysis(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [0]:
# from transformers import get_linear_schedule_with_warmup
# from transformers import AdamW
# import random
# seed_val = 42
# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)

# def train(args):
#     optimizer = AdamW(model.parameters(),
#                   lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
#                   eps = 1e-8, # args.adam_epsilon  - default is 1e-8.
#                   weight_decay = 0.1
#                 )
#     epochs = 10
#     scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                         num_warmup_steps = 0, 
#                                         num_training_steps = total_steps)
#     loss_values
#     for epoch_i in range(0, epochs):
#         t0 = time.time()
#         total_loss = 0
#         model.train()
        



In [0]:

# args = AttrDict()
# args_dict = {
#               'gpu':True, 
#               'valid':False, 
#               'checkpoint':"", 
#             #   'batch_size':32, 
#               'epochs':20, 
#               'seed':8,
# }